Import Required Libraries: You import the necessary libraries for building and training your models.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

Hyperparameters: You define the model’s hyperparameters such as input/output dimensions, learning rate, etc.

In [ ]:

num_epochs = 100
batch_size = 64
learning_rate = 0.001
num_symbols = 16
num_bits_per_symbol = 4
channel_noise_std = 0.1

Define Variables: Define variables specific to your communication system, like SNR and noise.

In [ ]:
input_dim = num_bits_per_symbol
output_dim = num_symbols

Transmitter Architecture: A neural network that encodes the input signal into a transmitted signal.

In [ ]:
def build_transmitter(input_dim, output_dim):
model = models.Sequential()
model.add(layers.InputLayer(input_shape=(input_dim,)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(output_dim, activation='softmax'))
return model

Declaring Models: The models are declared and connected to simulate the transmission process.

In [ ]:
# Input and Transmitter model
transmitter = build_transmitter(input_dim, output_dim)
receiver = build_receiver(output_dim, input_dim)

compiling the models

In [ ]:
# Compile the models
transmitter.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss='categorical_crossentropy')
receiver.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss='categorical_crossentropy')

Receiver Architecture: Another neural network that decodes the received signal back into the original input.

In [ ]:
def build_receiver(output_dim, input_dim):
 model = models.Sequential()
model.add(layers.InputLayer(input_shape=(output_dim,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(input_dim, activation='softmax'))
return model

Alternative Training: Both transmitter and receiver are trained iteratively.

In [ ]:
transmitter_losses = []
eceiver_losses = []
for epoch in range(num_epochs):
    # Generate random data
data = np.random.randint(0, 2, (batch_size, input_dim))
labels = tf.keras.utils.to_categorical(data, num_classes=output_dim)

 # Train transmitter
transmitter_loss = transmitter.train_on_batch(data, labels)
transmitter_losses.append(transmitter_loss)

 Pass through channel (add noise)
transmitted_signal = transmitter.predict(data)
 noisy_signal = transmitted_signal + np.random.normal(0, channel_noise_std, transmitted_signal.shape)

    # Train receiver
  receiver_loss = receiver.train_on_batch(noisy_signal, labels)
 receiver_losses.append(receiver_loss)



Plotting: The training losses for both transmitter and receiver are plotted to monitor progress.

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(transmitter_losses, label='Transmitter Loss')
plt.plot(receiver_losses, label='Receiver Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Transmitter and Receiver Losses')
plt.show()


Predictions: Finally, the trained models are used to make predictions on test data, and accuracy is calculated.

In [ ]:
# Predictions
test_data = np.random.randint(0, 2, (10, input_dim))
transmitted_signal = transmitter.predict(test_data)
noisy_signal = transmitted_signal + np.random.normal(0, channel_noise_std, transmitted_signal.shape)
received_signal = receiver.predict(noisy_signal)

print("Test Data:")
print(test_data)
print("Received Signal:")
print(np.argmax(received_signal, axis=1))